# Simple Dataset Demo (v3.x)

Embedded audio in parquet. Uses HuggingFace `datasets` v3.x with `trust_remote_code=True`.

**Note**: The simple dataset works the same in both v3.x and v4.x since it uses parquet with embedded audio.

*Generated by Claude*

In [ ]:
# Check datasets version
from oyez_sa_asr.hf_compat import datasets_version, supports_loading_scripts

print(f"datasets version: {datasets_version()}")
print(f"Supports loading scripts: {supports_loading_scripts()}")

In [ ]:
# v3.x loading with trust_remote_code (optional for simple)
from datasets import Audio, load_dataset

# Use decode=False to avoid torchcodec requirement on some platforms
ds = load_dataset("../datasets/simple", "lt1m", trust_remote_code=True)
ds = ds.cast_column("audio", Audio(decode=False))
sample = ds["train"][0]

print(f"Sentence: {sample['sentence'][:100]}...")
print(f"Speaker: {sample.get('speaker', 'N/A')}")
print(f"Duration: {sample.get('duration', 'N/A')}s")

In [ ]:
import io

from IPython.display import Audio as IPAudio

# Audio is embedded as bytes - decode with soundfile/av
audio_bytes = sample["audio"]["bytes"]

# Try soundfile first, fall back to av
try:
    import soundfile as sf

    audio_array, sr = sf.read(io.BytesIO(audio_bytes))
except ImportError:
    import av
    import numpy as np

    container = av.open(io.BytesIO(audio_bytes))
    frames = [f.to_ndarray() for f in container.decode(audio=0)]
    audio_array = np.concatenate(frames, axis=1).flatten()
    sr = container.streams.audio[0].rate
    container.close()

print(f"Audio: {len(audio_array)} samples @ {sr} Hz")
IPAudio(data=audio_array, rate=sr)

## Available Splits

- `lt1m`: Utterances < 1 minute (most common)
- `lt5m`: Utterances 1-5 minutes
- `lt30m`: Utterances 5-30 minutes

In [ ]:
# Load different split
ds_5m = load_dataset("../datasets/simple", "lt5m", trust_remote_code=True)
ds_5m = ds_5m.cast_column("audio", Audio(decode=False))
sample_5m = ds_5m["train"][0]
print(f"lt5m sample duration: {sample_5m.get('duration', 'N/A')}s")

## Note on v3.x vs v4.x

For the `simple` dataset, both versions work identically since the audio is embedded in parquet files. The `trust_remote_code=True` parameter is optional.